In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re
from IPython.display import display, HTML
import ipywidgets as widgets


In [2]:
!pip install flask-ngrok

In [3]:
!pip install flask flask_ngrok pyterrier

In [4]:
!python app.py

python3: can't open file '/content/app.py': [Errno 2] No such file or directory


In [5]:
#install the Pyterrier framework
!pip install python-terrier
# install the nltk modules
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.2 MB/s eta 0:00:00
  Preparing metadata (setup.p

In [6]:
try:
    import pyterrier as pt
except ImportError:
    !pip install python-terrier
    import pyterrier as pt
# if not pt.started():
#   # In this lab, we need to specify that we start PyTerrier with PRF enabled
#   pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])
nltk.download("stopwords")
nltk.download("punkt")
stop=set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
if not pt.started():
  # In this lab, we need to specify that we start PyTerrier with PRF enabled
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



# Data collection

In [8]:
import pandas as pd
import os

# Function to ensure directory exists
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

# Function to convert JSONL to CSV
def jsonl_to_csv(jsonl_file_path, csv_file_path):
    ensure_dir(csv_file_path)
    if os.path.exists(jsonl_file_path) and os.path.getsize(jsonl_file_path) > 0:
        try:
            df = pd.read_json(jsonl_file_path, lines=True)
            df.to_csv(csv_file_path, index=False)
            print(f"Successfully converted {jsonl_file_path} to {csv_file_path}")
        except ValueError as e:
            print(f"Error reading {jsonl_file_path}: {str(e)}")
    else:
        print(f"File not found or is empty: {jsonl_file_path}")

# Function to convert TSV to CSV
def tsv_to_csv(tsv_file_path, csv_file_path):
    ensure_dir(csv_file_path)
    if os.path.exists(tsv_file_path) and os.path.getsize(tsv_file_path) > 0:
        try:
            df = pd.read_csv(tsv_file_path, sep='\t', encoding='utf-8')
            df.to_csv(csv_file_path, index=False)
            print(f"Successfully converted {tsv_file_path} to {csv_file_path}")
        except Exception as e:
            print(f"Failed to convert {tsv_file_path} to CSV: {str(e)}")
    else:
        print(f"File not found or is empty: {tsv_file_path}")

# Paths for JSONL files
corpus_jsonl_path = '/content/queries.jsonl'  # Verify this path
queries_jsonl_path = '/content/queries.jsonl'  # Verify this path

# Paths for TSV files
test_tsv_path = '/content/test.tsv'  # Verify this path
train_tsv_path = '/content/train.tsv'  # Verify this path

# Output CSV paths
corpus_csv_path = '/mnt/data/corpus.csv'
queries_csv_path = '/mnt/data/queries.csv'
test_csv_path = '/mnt/data/test.csv'
train_csv_path = '/mnt/data/train.csv'

# Convert files
jsonl_to_csv(corpus_jsonl_path, corpus_csv_path)
jsonl_to_csv(queries_jsonl_path, queries_csv_path)
tsv_to_csv(test_tsv_path, test_csv_path)
tsv_to_csv(train_tsv_path, train_csv_path)


Successfully converted /content/queries.jsonl to /mnt/data/corpus.csv
Successfully converted /content/queries.jsonl to /mnt/data/queries.csv
Successfully converted /content/test.tsv to /mnt/data/test.csv
Successfully converted /content/train.tsv to /mnt/data/train.csv


In [9]:
x=pd.read_csv("/mnt/data/corpus.csv")

In [10]:
vv=pd.read_csv("/mnt/data/test.csv")

In [11]:
vv

,query-id,corpus-id,score
0,PLAIN-2,MED-2427,2
1,PLAIN-2,MED-10,2
2,PLAIN-2,MED-2429,2
3,PLAIN-2,MED-2430,2
4,PLAIN-2,MED-2431,2
...,...,...,...
12329,PLAIN-3472,MED-3623,1
12330,PLAIN-3472,MED-3624,1
12331,PLAIN-3472,MED-3625,1
12332,PLAIN-3472,MED-3626,1


In [12]:
import pandas as pd

# Load the data
vv = pd.read_csv('/mnt/data/test.csv')

# Renaming the columns to match the other DataFrame
vv.columns = ['qid', 'docno', 'label']

# Convert 'qid' and 'docno' to integers by extracting numbers and converting
vv['qid'] = vv['qid']
vv['docno'] = vv['docno'].str.extract('(\d+)').astype(int)

# Explicitly convert 'label' to integer
vv['label'] = vv['label'].astype(int)
vv['qid'] = vv['label'].astype(str)
vv['docno'] = vv['docno'].astype(str)


# Check data types to ensure all are integers
print(vv.dtypes)

# Optionally, save the corrected DataFrame
vv.to_csv('/mnt/data/corrected_test.csv', index=False)  # Ensure the path is correct

# Display the first few rows to verify
print(vv.head())

qid      object
docno    object
label     int64
dtype: object
  qid docno  label
0   2  2427      2
1   2    10      2
2   2  2429      2
3   2  2430      2
4   2  2431      2


In [13]:
vv

,qid,docno,label
0,2,2427,2
1,2,10,2
2,2,2429,2
3,2,2430,2
4,2,2431,2
...,...,...,...
12329,1,3623,1
12330,1,3624,1
12331,1,3625,1
12332,1,3626,1


In [14]:
x

,_id,text,metadata
0,PLAIN-3,Breast Cancer Cells Feed on Cholesterol,{'url': 'http://nutritionfacts.org/2015/07/14/...
1,PLAIN-4,Using Diet to Treat Asthma and Eczema,{'url': 'http://nutritionfacts.org/2015/07/09/...
2,PLAIN-5,Treating Asthma With Plants vs. Pills,{'url': 'http://nutritionfacts.org/2015/07/07/...
3,PLAIN-6,How Fruits and Vegetables Can Treat Asthma,{'url': 'http://nutritionfacts.org/2015/07/02/...
4,PLAIN-7,How Fruits and Vegetables Can Prevent Asthma,{'url': 'http://nutritionfacts.org/2015/06/30/...
...,...,...,...
3232,PLAIN-3432,Healthy Chocolate Milkshakes,{'url': 'http://nutritionfacts.org/video/healt...
3233,PLAIN-3442,The Healthiest Vegetables,{'url': 'http://nutritionfacts.org/video/the-h...
3234,PLAIN-3452,Bowel Movement Frequency,{'url': 'http://nutritionfacts.org/video/bowel...
3235,PLAIN-3462,Olive Oil and Artery Function,{'url': 'http://nutritionfacts.org/video/olive...


In [15]:
vaswani_dataset = pt.datasets.get_dataset("vaswani")
topics = vaswani_dataset.get_topics()
topics.head(5)
qrels = vaswani_dataset.get_qrels()
qrels.dtypes

query-text.trec:   0%|          | 0.00/3.05k [00:00<?, ?iB/s]

qrels:   0%|          | 0.00/6.63k [00:00<?, ?iB/s]

qid      object
docno    object
label     int64
dtype: object

# preprocessing

In [16]:
# Initialize Porter Stemmer
stemmer = PorterStemmer()

stop_words = stopwords.words('english')

def preprocess(text):
    text = re.sub(r"http\S+", " ", text) # remove urls
    text = re.sub(r"RT ", " ", text) # remove rt
    text = re.sub(r"@[\w]*", " ", text) # remove handles
    text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
    text = re.sub(r'\t', ' ', text) # remove tabs
    text = re.sub(r'\n', ' ', text) # remove line jump
    text = re.sub(r'\s+', ' ', text).strip().lower()
    tokens = word_tokenize(text)
    return [stemmer.stem(token) for token in tokens if token not in stop_words]



# Indexing

In [17]:
# Load data
docs_df = pd.read_csv("/mnt/data/corpus.csv")
# Apply preprocessing to document content
docs_df['processed_content'] = docs_df['text'].apply(lambda x: ' '.join(preprocess(str(x))))
docs_df["docno"] = docs_df.index.astype(str)

# Build an inverted index

indexer = pt.DFIndexer("./index", overwrite=True)
index_ref = indexer.index(docs_df['processed_content'], docs_df['docno'])
index = pt.IndexFactory.of(index_ref)

22:41:53.386 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (358) - further warnings are suppressed
22:42:01.378 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 6 empty documents


In [18]:
docs_df

,_id,text,metadata,processed_content,docno
0,PLAIN-3,Breast Cancer Cells Feed on Cholesterol,{'url': 'http://nutritionfacts.org/2015/07/14/...,breast cancer cell feed cholesterol,0
1,PLAIN-4,Using Diet to Treat Asthma and Eczema,{'url': 'http://nutritionfacts.org/2015/07/09/...,use diet treat asthma eczema,1
2,PLAIN-5,Treating Asthma With Plants vs. Pills,{'url': 'http://nutritionfacts.org/2015/07/07/...,treat asthma plant vs pill,2
3,PLAIN-6,How Fruits and Vegetables Can Treat Asthma,{'url': 'http://nutritionfacts.org/2015/07/02/...,fruit veget treat asthma,3
4,PLAIN-7,How Fruits and Vegetables Can Prevent Asthma,{'url': 'http://nutritionfacts.org/2015/06/30/...,fruit veget prevent asthma,4
...,...,...,...,...,...
3232,PLAIN-3432,Healthy Chocolate Milkshakes,{'url': 'http://nutritionfacts.org/video/healt...,healthi chocol milkshak,3232
3233,PLAIN-3442,The Healthiest Vegetables,{'url': 'http://nutritionfacts.org/video/the-h...,healthiest veget,3233
3234,PLAIN-3452,Bowel Movement Frequency,{'url': 'http://nutritionfacts.org/video/bowel...,bowel movement frequenc,3234
3235,PLAIN-3462,Olive Oil and Artery Function,{'url': 'http://nutritionfacts.org/video/olive...,oliv oil arteri function,3235


# For each term, maintain a list of document
# maps each unique word (or term) to the
## documents that contain that word.

In [19]:
def tf(processed_docs):
    tf_dict = {}
    for doc_id, doc_text in enumerate(processed_docs):
        doc_tokens = doc_text.split()
        for term in doc_tokens:
            if term not in tf_dict:
                tf_dict[term] = {}
            if doc_id not in tf_dict[term]:
                tf_dict[term][doc_id] = 0
            tf_dict[term][doc_id] += 1
    return tf_dict

processed_docs = docs_df['processed_content'].tolist()
tf_dict = tf(processed_docs)
print(tf_dict)


{'breast': {0: 1, 59: 1, 70: 1, 85: 1, 96: 1, 124: 1, 125: 1, 129: 1, 130: 1, 156: 1, 157: 1, 158: 1, 159: 1, 216: 1, 218: 1, 238: 1, 252: 1, 303: 1, 323: 1, 324: 1, 547: 1, 548: 1, 549: 1, 550: 1, 551: 1, 859: 1, 1836: 1, 1852: 1, 1861: 1, 1889: 1, 1922: 1, 1923: 1, 1983: 1, 1994: 1, 2009: 1, 2037: 1, 2045: 1, 2047: 1, 2052: 1, 2053: 1, 2054: 1, 2080: 1, 2086: 1, 2087: 1, 2088: 1, 2089: 1, 2237: 1, 2239: 1, 2240: 1, 2241: 1, 2247: 1, 2248: 1, 2324: 1, 2325: 1, 2326: 1, 2386: 1, 2387: 1, 2488: 1, 2542: 1, 2612: 1, 2630: 1, 2659: 1, 2845: 1, 2855: 1, 2883: 1, 2914: 1, 2929: 1, 2941: 1, 2943: 1, 2983: 1, 3169: 1}, 'cancer': {0: 1, 11: 1, 12: 1, 14: 1, 21: 1, 48: 1, 59: 1, 70: 1, 71: 1, 73: 1, 75: 1, 85: 1, 96: 1, 97: 1, 118: 1, 122: 1, 129: 1, 130: 1, 133: 1, 156: 1, 157: 1, 158: 1, 159: 1, 170: 1, 178: 1, 183: 1, 215: 1, 216: 1, 218: 1, 222: 1, 238: 1, 252: 1, 256: 1, 263: 1, 278: 1, 322: 1, 323: 1, 324: 1, 382: 1, 388: 1, 506: 1, 547: 1, 548: 1, 588: 1, 589: 1, 590: 1, 591: 1, 626: 1, 

# Query Processing

In [20]:
def search(query, k=10):
    processed_query = preprocess(query)
    processed_query_str = " ".join(processed_query)
    tf = pt.BatchRetrieve(index, wmodel="TF_IDF")
    results = tf.transform(processed_query_str)
    # evaluation=pt.Evaluate(results,x)
    return results
ss=search("cancer")


<ipython-input-20-7fd625f9af64>:5: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  results = tf.transform(processed_query_str)


In [21]:
ss

,qid,docid,docno,rank,score,query
0,1,588,588,0,3.104187,cancer
1,1,506,506,1,2.560414,cancer
2,1,547,547,2,2.560414,cancer
3,1,590,590,3,2.560414,cancer
4,1,591,591,4,2.560414,cancer
...,...,...,...,...,...,...
172,1,3169,3169,172,1.505510,cancer
173,1,263,263,173,1.364921,cancer
174,1,2037,2037,174,1.364921,cancer
175,1,2630,2630,175,1.364921,cancer


In [22]:
pt.Evaluate(ss,vv)

{'map': 0.006899993805621145, 'ndcg': 0.023783294532703208}

In [23]:
meta = index.getMetaIndex()
inv = index.getInvertedIndex()
lex = index.getLexicon()

# List of terms to retrieve document frequency
terms = preprocess("Breast Using Diet")  # Add your terms here

for term in terms:
    le = lex.getLexiconEntry(term)
    if le is None:
        print(f"Term '{term}' not found in the lexicon.")
        continue

    # Access the inverted index posting list for the term
    for posting in inv.getPostings(le):
        docno = meta.getItem("docno", posting.getId())
        print(f"Document: {docno}, Frequency: {posting.getFrequency()} for term: {term}")


Document: 0, Frequency: 1 for term: breast
Document: 59, Frequency: 1 for term: breast
Document: 70, Frequency: 1 for term: breast
Document: 85, Frequency: 1 for term: breast
Document: 96, Frequency: 1 for term: breast
Document: 124, Frequency: 1 for term: breast
Document: 125, Frequency: 1 for term: breast
Document: 129, Frequency: 1 for term: breast
Document: 130, Frequency: 1 for term: breast
Document: 156, Frequency: 1 for term: breast
Document: 157, Frequency: 1 for term: breast
Document: 158, Frequency: 1 for term: breast
Document: 159, Frequency: 1 for term: breast
Document: 216, Frequency: 1 for term: breast
Document: 218, Frequency: 1 for term: breast
Document: 238, Frequency: 1 for term: breast
Document: 252, Frequency: 1 for term: breast
Document: 303, Frequency: 1 for term: breast
Document: 323, Frequency: 1 for term: breast
Document: 324, Frequency: 1 for term: breast
Document: 547, Frequency: 1 for term: breast
Document: 548, Frequency: 1 for term: breast
Document: 549, F

In [24]:
def Expand_query(query, k=10):
    processed_query = preprocess(query)
    # Convert list of terms back to a string for searching
    processed_query_str = " ".join(processed_query)
    tf = pt.BatchRetrieve(index, wmodel="Tf")
    rm3_expander = pt.rewrite.RM3(index, fb_terms=10, fb_docs=100)
    rm3_qe = tf >> rm3_expander
    expanded_query = rm3_qe.search(query).iloc[0]["query"]
    expanded_terms = [term.split('^')[0] for term in expanded_query.split()[1:]]  # Extract only the terms
    return expanded_terms

expanded_terms = Expand_query("Breast Using Diet")
print(expanded_terms)


['breast', 'treat', 'mediterranean', 'prevent', 'soi', 'plant', 'ba', 'cancer', 'disea', 'diet']


In [33]:
tf =pt.BatchRetrieve(index, wmodel="TF_IDF")
x=tf.search("Breast Cancer")

In [34]:
x

,qid,docid,docno,rank,score,query
0,1,547,547,0,5.856264,Breast Cancer
1,1,96,96,1,4.983258,Breast Cancer
2,1,158,158,2,4.983258,Breast Cancer
3,1,548,548,3,4.983258,Breast Cancer
4,1,1889,1889,4,4.983258,Breast Cancer
...,...,...,...,...,...,...
193,1,2046,2046,193,1.489334,Breast Cancer
194,1,2125,2125,194,1.489334,Breast Cancer
195,1,2590,2590,195,1.489334,Breast Cancer
196,1,263,263,196,1.350250,Breast Cancer


In [27]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd

# Function to calculate cosine similarity
def cosine_similarity(v1, v2):
    # Ensure the embeddings are summed across all tokens and reduced to a 1D array
    sum_v1 = np.sum(v1, axis=0)
    sum_v2 = np.sum(v2, axis=0)
    if sum_v1.ndim > 1:
        sum_v1 = np.sum(sum_v1, axis=0)
    if sum_v2.ndim > 1:
        sum_v2 = np.sum(sum_v2, axis=0)

    dot_product = np.dot(sum_v1, sum_v2)
    norm_v1 = np.linalg.norm(sum_v1)
    norm_v2 = np.linalg.norm(sum_v2)
    return dot_product / (norm_v1 * norm_v2)

# Load the ELMo model
elmo = hub.load("https://tfhub.dev/google/elmo/3")

# Define the query sentence
query_sentence = "biomaterials"

# Assuming docs_df is your DataFrame with a 'processed_content' column
# Limit the loop to the first five rows
similarities = []
for doc_content in docs_df['processed_content'].iloc[:5]:  # Process only the first five documents
    # Generate ELMo embeddings for the query sentence and the document content
    query_embedding = elmo.signatures["default"](tf.constant([query_sentence]))["elmo"].numpy()
    doc_embedding = elmo.signatures["default"](tf.constant([doc_content]))["elmo"].numpy()

    # Calculate cosine similarity between the query and the document
    similarity_score = cosine_similarity(query_embedding, doc_embedding)
    similarities.append(similarity_score)

# Create a DataFrame to store similarity scores with corresponding documents
similarity_df = pd.DataFrame({
    'Document': docs_df['processed_content'].iloc[:5],  # Include only the first five documents
    'Similarity Score': similarities
})

# Rank the documents based on similarity scores
similarity_df = similarity_df.sort_values(by='Similarity Score', ascending=False)

# Print the ranked DataFrame
print(similarity_df)


                              Document  Similarity Score
0  breast cancer cell feed cholesterol          0.458735
3             fruit veget treat asthma          0.457064
4           fruit veget prevent asthma          0.447965
2           treat asthma plant vs pill          0.433618
1         use diet treat asthma eczema          0.418802


# Bert

In [28]:
import pandas as pd
import pyterrier as pt
if not pt.started():
    pt.init()

from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Define a simple preprocessing function, replace with your specific preprocessing needs
def preprocess(text):
    # Placeholder for text preprocessing, e.g., tokenization, lowercasing, etc.
    return text.lower().split()

# Load and preprocess the documents
docs_df = pd.read_csv("/mnt/data/corpus.csv")
docs_df['processed_content'] = docs_df['text'].apply(lambda x: ' '.join(preprocess(str(x))))
docs_df["docno"] = docs_df.index.astype(str)

# Build an inverted index
indexer = pt.DFIndexer("./index", overwrite=True)
index_ref = indexer.index(docs_df['processed_content'], docs_df['docno'])
index = pt.IndexFactory.of(index_ref)

# Function to expand a query using PyTerrier's RM3
def expand_query(query, index, k=10):
    retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"})
    rm3_expander = pt.rewrite.RM3(index, fb_terms=10, fb_docs=100)
    rm3_qe = retr >> rm3_expander
    expanded_query = rm3_qe.search(query)
    print('Expanded Query:', expanded_query)
    return expanded_query

# Sample query
query = "Breast Cancer"

# Expand the query
expanded_query_df = expand_query(query, index)

# Assuming 'expanded_query_df' contains a column 'query' with the expanded query terms
expanded_query_terms = " ".join(expanded_query_df["query"].tolist())

# Perform retrieval and reranking
retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"})
res = retr.search(expanded_query_terms)
result_merged = pd.merge(res, docs_df, on="docno", how="left")

# Sort by TF-IDF score
result_merged = result_merged.sort_values(by="score", ascending=False)
print('Before BERT:\n', result_merged.head())

# Load BERT model for reranking
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

# Prepare the input for BERT
inputs = tokenizer(result_merged["processed_content"].tolist(), padding=True, truncation=True, return_tensors="pt")

# Get scores from BERT
outputs = model(**inputs)
scores = outputs.logits[:, 1]  # Assuming binary classification and index 1 is the relevant class

# Process BERT output
reviews_result_v2 = pd.DataFrame({
    'preprocessed_title': result_merged['processed_content'],
    'score': scores.detach().numpy()
}).sort_values(by="score", ascending=False)
print('After BERT and expansion:\n', reviews_result_v2.head())


22:42:41.255 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (1128) - further warnings are suppressed
22:42:43.674 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 2 empty documents
Expanded Query:   qid        query_0                                              query
0   1  Breast Cancer  applypipeline:off breast^0.419990540 prevent^0...
Before BERT:
   qid  docid docno  rank     score  \
0   1    547   547     0  5.329982   
1   1    548   548     1  4.705996   
2   1   2542  2542     2  4.639654   
3   1   2089  2089     3  4.576359   
4   1     96    96     4  4.536645   

                                               query         _id  \
0  applypipeline:off breast^0.419990540 prevent^0...   PLAIN-755   
1  applypipeline:off breast^0.419990540 prevent^0...   PLAIN-756   
2  applypipeline:off breast^0.419990540 prevent^0...  PLAIN-3415   
3  applypipeline:off breast^0.419990540 prevent^0...  PLAIN-2816   
4  applypipeline:o

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


After BERT and expansion:
                                preprocessed_title     score
19              flax and breast cancer prevention  0.315321
145  vegan men: more testosterone but less cancer  0.309555
271     prevent glaucoma and see 27 miles farther  0.300505
168      raw broccoli and bladder cancer survival  0.284949
29    why do asian women have less breast cancer?  0.274314


In [29]:
!pip install pyngrok


In [30]:
from flask import Flask, request, render_template_string, send_file
import pandas as pd
import pyterrier as pt
from pyngrok import ngrok
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import socket
import time

# Initialize Flask app
app = Flask(__name__)

# Set up ngrok with your auth token
ngrok.set_auth_token('2g8zCKxXqTMHmP0xoncWWeVrial_2ZWJzQp1gbkEHx9foBSq2')  # Replace with your actual ngrok auth token

# Initialize PyTerrier
if not pt.started():
    pt.init()

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Initialize Porter Stemmer
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Define a simple preprocessing function
def preprocess(text):
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"@\w*", " ", text)
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip().lower()
    tokens = word_tokenize(text)
    return ' '.join([stemmer.stem(token) for token in tokens if token not in stop_words])

# Load and preprocess the documents
docs_df = pd.read_csv("/mnt/data/corpus.csv")
docs_df['processed_content'] = docs_df['text'].apply(lambda x: ''.join(preprocess(str(x))))
docs_df["docno"] = docs_df.index.astype(str)

# Create and index documents
indexer = pt.DFIndexer("./index", overwrite=True)
index_ref = indexer.index(docs_df['processed_content'], docs_df['docno'])
index = pt.IndexFactory.of(index_ref)

# Load BERT model and tokenizer once during initialization
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

# Function to expand a query using PyTerrier's RM3
def expand_query(query, index):
    retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"})
    rm3_expander = pt.rewrite.RM3(index, fb_terms=10, fb_docs=100)
    rm3_qe = retr >> rm3_expander
    expanded_query = rm3_qe.search(query)
    expanded_query_terms = " ".join(expanded_query["query"].tolist())
    return expanded_query_terms

# Define the new search function using BERT for reranking
def search(query, k=10, bert_batch_size=32):
    start_time = time.time()

    # Preprocess the query
    processed_query = "".join(preprocess(query))

    # Expand the query
    expanded_query_terms = expand_query(processed_query, index)

    # Perform retrieval
    retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"})
    res = retr.search(expanded_query_terms)

    # Merge results with original docs
    result_merged = pd.merge(res, docs_df, on="docno", how="left")

    # Sort by TF-IDF score and take top k results for BERT reranking
    top_k_results = result_merged.sort_values(by="score", ascending=False).head(k)

    # Prepare the input for BERT
    inputs = tokenizer(top_k_results["processed_content"].tolist(), padding=True, truncation=True, return_tensors="pt", max_length=512)

    # Get scores from BERT
    with torch.no_grad():
        outputs = model(**inputs, return_dict=True)
    scores = outputs.logits[:, 1]  # Assuming binary classification and index 1 is the relevant class

    # Process BERT output and sort results
    top_k_results['bert_score'] = scores.detach().cpu().numpy()
    top_k_results = top_k_results.sort_values(by="bert_score", ascending=False)

    end_time = time.time()
    effectiveness = round(end_time - start_time, 2)
    efficiency = len(top_k_results)

    return top_k_results[["docno", "bert_score"]].values.tolist(), efficiency, effectiveness

# HTML Template
HTML_TEMPLATE = """
<!doctype html>
<html lang="en">
<head>
    <meta charset="utf-8">
    <title>Hospital Search Engine</title>
    <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap" rel="stylesheet">
    <style>
        body {
            font-family: 'Poppins', sans-serif;
            background: url('/background_image') no-repeat center center fixed;
            background-size: cover;
            margin: 0;
            padding: 20px;
            color: #333;
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: flex-start; /* Align content at the top */
        }
        h1 {
            color: #000000; /* Changed text color to black */
            text-align: center;
            margin-top: 80px; /* Reduced top margin */
            margin-bottom: 20px; /* Reduced margin bottom */
        }
        form {
            background-color: rgba(255, 255, 255, 0.9);
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.1);
            display: flex;
            flex-direction: column;
            align-items: center;
            margin-bottom: 40px; /* Increased margin bottom */
            width: 100%;
            max-width: 600px;
        }
        input[type="text"], input[type="submit"] {
            width: 100%;
            padding: 12px;
            border-radius: 4px;
            border: 1px solid #ccc;
            box-sizing: border-box;
            font-size: 16px;
        }
        input[type="text"] {
            margin-bottom: 10px; /* Reduced margin bottom */
        }
        input[type="submit"] {
            background-color: #007bff;
            color: white;
            border: none;
            cursor: pointer;
            font-weight: 600;
            transition: background-color 0.3s ease;
        }
        input[type="submit"]:hover {
            background-color: #0056b3;
        }
        .results-container {
            width: 100%;
            max-width: 800px;

        }
        .results-header {
            text-align: center;
            margin-bottom: 20px;
            color: #000000; /* Changed text color to black */
        }
        ul {
            list-style-type: none;
            padding: 0;

        }
        li {
            background-color: rgba(255, 255, 255, 0.9);
            padding: 10px;
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            margin-bottom: 10px;
        }
        a {
            color: #000000;
            text-decoration: none;
            font-weight: 600;
            transition: color 0.3s ease;
        }
        a:hover {
            text-decoration: underline;
            color: #0056b3;
        }
        .document-content {
            padding: 20px;
            background-color: rgba(255, 255, 255, 0.9);
            border-radius: 8px;
            box-shadow: 0 2px 4px rgba(0,0,0,0.1);
            margin-bottom: 20px;
            max-width: 800px;
            width: 100%;
        }
        @media (max-width: 768px) {
            body {
                padding: 10px;
            }
            form, .results-container {
                width: calc(100% - 20px);
            }
            input[type="text"], input[type="submit"] {
                width: 100%;
            }
        }
    </style>
</head>
<body>
    <h1>Hospital Search Engine Interface</h1>
    <form action="/search" method="post">
        <input type="text" name="query" placeholder="Enter search terms..." required>
        <input type="submit" value="Search">
    </form>
    <div class="results-container">
        {% if query %}
        <div class="results-header">
            <h2>Results for "{{ query }}":</h2>
            <p>About {{ efficiency }} results ({{ effectiveness }} seconds)</p>
        </div>
        {% if results %}
        <ul>
            {% for doc_id, bert_score in results %}
            <li><a href="/document/{{ doc_id }}" target="_blank">Document ID: {{ doc_id }}</a></li>
            {% endfor %}
        </ul>
        {% else %}
        <p>No results found.</p>
        {% endif %}
        {% endif %}
    </div>
</body>
</html>

"""

@app.route("/", methods=["GET"])
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route("/search", methods=["POST"])
def search_route():
    query = request.form['query']
    results, efficiency, effectiveness = search(query)
    return render_template_string(HTML_TEMPLATE, query=query, results=results, efficiency=efficiency, effectiveness=effectiveness)

@app.route("/document/<doc_id>", methods=["GET"])
def document_view(doc_id):
    document_content = docs_df[docs_df['docno'] == doc_id]['text'].iloc[0]
    content_html = f"<div class='document-content'>{document_content}</div>"
    return render_template_string(f"""
    <!doctype html>
    <html lang="en">
    <head>
        <meta charset="utf-8">
        <title>Document {doc_id}</title>
        <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@400;600&display=swap" rel="stylesheet">
        <style>
            body {{
                font-family: 'Poppins', sans-serif;
                background: url('/background_image') no-repeat center center fixed;
                background-size: cover;
                margin: 0;
                padding: 20px;
                color: #333;
                display: flex;
                flex-direction: column;
                align-items: center;
            }}
            .document-content {{
                padding: 20px;
                background-color: rgba(255, 255, 255, 0.9);
                border-radius: 8px;
                box-shadow: 0 2px 4px rgba(0,0,0,0.1);
                margin-bottom: 20px;
                max-width: 800px;
                width: 100%;
            }}
        </style>
    </head>
    <body>
        <h1>Document ID: {doc_id}</h1>
        {content_html}
    </body>
    </html>
    """)

@app.route("/background_image")
def background_image():
    return send_file("/content/interior-modern-emergency-room-with-empty-nurses-station-generative-ai_587448-2137.avif", mimetype='image/avif')




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


22:46:28.629 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (358) - further warnings are suppressed
22:46:32.616 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 6 empty documents


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def get_open_port():
    """Find an available port on localhost for ngrok and Flask to use."""
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.bind(("", 0))
    port = sock.getsockname()[1]
    sock.close()
    return port

if __name__ == "__main__":
    port = get_open_port()
    ngrok_tunnel = ngrok.connect(port)
    print('NGROK Tunnel URL:', ngrok_tunnel.public_url)
    app.run(port=port)